EDA

In [23]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', 50)
import plotly.express as px


In [24]:
df_train = pd.read_csv("./data/GUIDE_Train.csv")
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9516837 entries, 0 to 9516836
Data columns (total 45 columns):
 #   Column              Dtype  
---  ------              -----  
 0   Id                  int64  
 1   OrgId               int64  
 2   IncidentId          int64  
 3   AlertId             int64  
 4   Timestamp           object 
 5   DetectorId          int64  
 6   AlertTitle          int64  
 7   Category            object 
 8   MitreTechniques     object 
 9   IncidentGrade       object 
 10  ActionGrouped       object 
 11  ActionGranular      object 
 12  EntityType          object 
 13  EvidenceRole        object 
 14  DeviceId            int64  
 15  Sha256              int64  
 16  IpAddress           int64  
 17  Url                 int64  
 18  AccountSid          int64  
 19  AccountUpn          int64  
 20  AccountObjectId     int64  
 21  AccountName         int64  
 22  DeviceName          int64  
 23  NetworkMessageId    int64  
 24  EmailClusterId      floa

In [25]:


incident_counts = df_train['IncidentGrade'].value_counts().reset_index()
incident_counts.columns = ['IncidentGrade', 'Count']

fig = px.pie(
    incident_counts,
    names='IncidentGrade',
    values='Count',
    title='Target Variable Distribution (Incident Grade)',
    color_discrete_sequence=px.colors.qualitative.Pastel 
)

fig.update_traces(
    textposition='inside', 
    textinfo='percent+label',
    pull=[0.1 if name == 'FalsePositive' else 0 for name in incident_counts['IncidentGrade']] 
)

fig.update_layout(
    title_x=0.5, 
    font=dict(size=12)
)

fig.show()

In [26]:
# 1. Count and sort null values in descending order
null_counts = df_train.isnull().sum()
null_counts = null_counts[null_counts > 0].sort_values(ascending=False)

if null_counts.empty:
    print("✅ There are no missing (null) values in the dataframe.")
else:
    print("Missing Values by Column (Descending Order):")
    # Display the sorted null counts
    print(null_counts.to_markdown(numalign="left", stralign="left"))

    # Create a DataFrame for plotting
    plot_df = pd.DataFrame({
        'Column Name': null_counts.index,
        'Missing Values Count': null_counts.values
    })

    # 2. Create the HORIZONTAL bar chart
    fig = px.bar(
        plot_df,
        # Swap x and y for a horizontal chart
        x='Missing Values Count',
        y='Column Name',
        title='Missing (Null) Values Per Column (Descending Order) - Horizontal Bar Chart',
        labels={'Missing Values Count': 'Number of Missing Values', 'Column Name': 'Feature Column'},
        color='Missing Values Count', # Color the bars by count
        color_continuous_scale=px.colors.sequential.Viridis
    )

    # Ensure the highest count is at the top by using 'total ascending' on the y-axis
    # This sorts the bars based on the x-value (Missing Values Count)
    fig.update_yaxes(categoryorder='total ascending')

    # Customize the layout for better readability
    fig.update_layout(
        plot_bgcolor='white',
        margin=dict(t=50, b=50, l=150),
        xaxis_title_font_size=14,
        yaxis_title_font_size=14
    )

    fig.show()

Missing Values by Column (Descending Order):
|                   | 0           |
|:------------------|:------------|
| ResourceType      | 9.50976e+06 |
| ActionGranular    | 9.46077e+06 |
| ActionGrouped     | 9.46077e+06 |
| ThreatFamily      | 9.44196e+06 |
| EmailClusterId    | 9.42002e+06 |
| AntispamDirection | 9.33954e+06 |
| Roles             | 9.29869e+06 |
| SuspicionLevel    | 8.07271e+06 |
| LastVerdict       | 7.28257e+06 |
| MitreTechniques   | 5.46839e+06 |
| IncidentGrade     | 51340       |


In [27]:
initial_shape = df_train.shape

cols_to_drop = [
    'Id',
    'ResourceType',
    'ActionGranular',
    'ActionGrouped',
    'ThreatFamily',
    'EmailClusterId',
    'AntispamDirection',
    'Roles',
    'SuspicionLevel',
    'LastVerdict',
    'MitreTechniques',
    'IncidentId'
]

df_train_cleaned = df_train.drop(columns=cols_to_drop)
rows_before_dropna = df_train_cleaned.shape[0]

df_train_cleaned = df_train_cleaned.dropna(subset=['IncidentGrade'])

final_shape = df_train_cleaned.shape
rows_dropped = rows_before_dropna - final_shape[0]

# 1. Count and sort null values in descending order
null_counts = df_train_cleaned.isnull().sum()
null_counts = null_counts[null_counts > 0].sort_values(ascending=False)

 

print("--- Cleaning Summary ---")
print(f"Original DataFrame shape: {initial_shape}")
print(f"Number of rows dropped due to NaN in IncidentGrade: {rows_dropped}")
print(f"Final DataFrame shape after cleaning: {final_shape}")
del df_train


--- Cleaning Summary ---
Original DataFrame shape: (9516837, 45)
Number of rows dropped due to NaN in IncidentGrade: 51340
Final DataFrame shape after cleaning: (9465497, 33)


In [6]:
df_train_cleaned.head()

,OrgId,AlertId,Timestamp,DetectorId,AlertTitle,Category,IncidentGrade,EntityType,EvidenceRole,DeviceId,Sha256,IpAddress,Url,AccountSid,AccountUpn,AccountObjectId,AccountName,DeviceName,NetworkMessageId,RegistryKey,RegistryValueName,RegistryValueData,ApplicationId,ApplicationName,OAuthApplicationId,FileName,FolderPath,ResourceIdName,OSFamily,OSVersion,CountryCode,State,City
0,0,123247,2024-06-04T06:05:15.000Z,7,6,InitialAccess,TruePositive,Ip,Related,98799,138268,27,160396,441377,673934,425863,453297,153085,529644,1631,635,860,2251,3421,881,289573,117668,3586,5,66,31,6,3
1,88,210035,2024-06-14T03:01:25.000Z,58,43,Exfiltration,FalsePositive,User,Impacted,98799,138268,360606,160396,22406,23032,22795,24887,153085,529644,1631,635,860,2251,3421,881,289573,117668,3586,5,66,242,1445,10630
2,809,712507,2024-06-13T04:52:55.000Z,423,298,InitialAccess,FalsePositive,Url,Related,98799,138268,360606,68652,441377,673934,425863,453297,153085,529644,1631,635,860,2251,3421,881,289573,117668,3586,5,66,242,1445,10630
3,92,774301,2024-06-10T16:39:36.000Z,2,2,CommandAndControl,BenignPositive,Url,Related,98799,138268,360606,13,441377,673934,425863,453297,153085,529644,1631,635,860,2251,3421,881,289573,117668,3586,5,66,242,1445,10630
4,148,188041,2024-06-15T01:08:07.000Z,9,74,Execution,TruePositive,User,Impacted,98799,138268,360606,160396,449,592,440,479,153085,529644,1631,635,860,2251,3421,881,289573,117668,3586,5,66,242,1445,10630


In [7]:
df_train_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9465497 entries, 0 to 9516836
Data columns (total 33 columns):
 #   Column              Dtype 
---  ------              ----- 
 0   OrgId               int64 
 1   AlertId             int64 
 2   Timestamp           object
 3   DetectorId          int64 
 4   AlertTitle          int64 
 5   Category            object
 6   IncidentGrade       object
 7   EntityType          object
 8   EvidenceRole        object
 9   DeviceId            int64 
 10  Sha256              int64 
 11  IpAddress           int64 
 12  Url                 int64 
 13  AccountSid          int64 
 14  AccountUpn          int64 
 15  AccountObjectId     int64 
 16  AccountName         int64 
 17  DeviceName          int64 
 18  NetworkMessageId    int64 
 19  RegistryKey         int64 
 20  RegistryValueName   int64 
 21  RegistryValueData   int64 
 22  ApplicationId       int64 
 23  ApplicationName     int64 
 24  OAuthApplicationId  int64 
 25  FileName            int

In [28]:
# List of columns that are nominal (categorical) but stored as integers
nominal_int_cols = [
     'OrgId',  'AlertId', 'DetectorId', 'AlertTitle', 'DeviceId',
    'Sha256', 'IpAddress', 'Url', 'AccountSid', 'AccountUpn', 'AccountObjectId', 
    'AccountName', 'DeviceName', 'NetworkMessageId', 'RegistryKey', 'RegistryValueName',
    'RegistryValueData', 'ApplicationId', 'ApplicationName', 'OAuthApplicationId', 
    'FileName', 'FolderPath', 'ResourceIdName', 'OSFamily', 'OSVersion', 
    'CountryCode', 'State', 'City', 'Category', 'EntityType','EvidenceRole'
]

# Convert nominal integer columns to 'category'
df_train_cleaned[nominal_int_cols] = df_train_cleaned[nominal_int_cols].astype('category')

# Convert the Timestamp column to datetime
df_train_cleaned['Timestamp'] = pd.to_datetime(df_train_cleaned['Timestamp'])

df_train_cleaned.info()


<class 'pandas.core.frame.DataFrame'>
Index: 9465497 entries, 0 to 9516836
Data columns (total 33 columns):
 #   Column              Dtype              
---  ------              -----              
 0   OrgId               category           
 1   AlertId             category           
 2   Timestamp           datetime64[ns, UTC]
 3   DetectorId          category           
 4   AlertTitle          category           
 5   Category            category           
 6   IncidentGrade       object             
 7   EntityType          category           
 8   EvidenceRole        category           
 9   DeviceId            category           
 10  Sha256              category           
 11  IpAddress           category           
 12  Url                 category           
 13  AccountSid          category           
 14  AccountUpn          category           
 15  AccountObjectId     category           
 16  AccountName         category           
 17  DeviceName          category    

In [29]:
df_train_cleaned['Hour'] = df_train_cleaned['Timestamp'].dt.hour.astype('category')
df_train_cleaned['DayOfWeek'] = df_train_cleaned['Timestamp'].dt.dayofweek.astype('category') # Monday=0, Sunday=6
df_train_cleaned['Month'] = df_train_cleaned['Timestamp'].dt.month.astype('category')


df_train_cleaned['IsBusinessHours'] = df_train_cleaned['Hour'].apply(lambda x: 1 if 8 <= x < 18 else 0).astype('category')

df_train_cleaned.drop(columns=['Timestamp'], inplace=True)

print("✅ Successfully engineered four new time-based features.")
print(df_train_cleaned[['Hour', 'DayOfWeek', 'IsBusinessHours']].head().to_markdown(index=False, numalign="left", stralign="left"))
print(f"\nFinal DataFrame shape: {df_train_cleaned.shape}")

✅ Successfully engineered four new time-based features.
| Hour   | DayOfWeek   | IsBusinessHours   |
|:-------|:------------|:------------------|
| 6      | 1           | 0                 |
| 3      | 4           | 0                 |
| 4      | 3           | 0                 |
| 16     | 0           | 1                 |
| 1      | 5           | 0                 |

Final DataFrame shape: (9465497, 36)


In [30]:
unique_counts = df_train_cleaned.nunique().sort_values(ascending=False)

# Create a DataFrame for display
unique_counts_df = pd.DataFrame({
    'Column Name': unique_counts.index,
    'Unique Count (Cardinality)': unique_counts.values
})

print("--- Column Cardinality (Unique Value Counts) ---")
print(unique_counts_df.to_markdown(index=False, numalign="left", stralign="left"))

--- Column Cardinality (Unique Value Counts) ---
| Column Name        | Unique Count (Cardinality)   |
|:-------------------|:-----------------------------|
| AlertId            | 1248789                      |
| AccountUpn         | 524760                       |
| NetworkMessageId   | 375196                       |
| AccountName        | 363254                       |
| AccountSid         | 353113                       |
| AccountObjectId    | 338252                       |
| IpAddress          | 285957                       |
| FileName           | 222085                       |
| Url                | 123252                       |
| DeviceName         | 114267                       |
| Sha256             | 106416                       |
| FolderPath         | 87832                        |
| AlertTitle         | 79952                        |
| DeviceId           | 75557                        |
| City               | 9342                         |
| DetectorId         | 7813      

Feature Engenring

In [ ]:
from sklearn.preprocessing import LabelEncoder
import numpy as np
import joblib
import gc

# --- Define Global Variables for the Pipeline ---

# These are the lists determined during your EDA:
OHE_COLS = [
    'Hour', 'DayOfWeek', 'Month', 'IsBusinessHours', 'Category', 
    'OSFamily', 'EntityType', 'EvidenceRole'
]

# Initial columns to drop due to high null count/uniqueness
COLS_TO_DROP_INITIAL = [
    'ResourceType', 'ActionGranular', 'ActionGrouped', 'ThreatFamily', 
    'EmailClusterId', 'AntispamDirection', 'Roles', 'SuspicionLevel', 
    'LastVerdict', 'MitreTechniques', 'Id', 'IncidentId'
]

# --- 1. The Core Transformation Function ---

def transform_data(df_train, df_val):
    """
    Performs all cleaning and feature engineering steps on both datasets.
    Calculates frequency maps from df_train and applies them to both.
    """
    
    # Dictionary to store artifacts calculated from training data
    freq_maps = {}
    
    # --- Initial Cleaning and Temporal Feature Extraction ---
    def initial_clean(df):
        df = df.copy()
        # Drop columns absent in the final feature set
        df = df.drop(columns=[col for col in COLS_TO_DROP_INITIAL if col in df.columns], errors='ignore')
        
        # Drop rows with NaN in the target variable
        df = df.dropna(subset=['IncidentGrade'])

        # Handle 'Usage' column unique to validation (if present)
        if 'Usage' in df.columns:
            df = df.drop(columns=['Usage'])

        # Temporal Features (Corrected)
        df['Timestamp'] = pd.to_datetime(df['Timestamp'])
        
        # Extract integer hours and days
        df['Hour'] = df['Timestamp'].dt.hour.astype('category')
        df['DayOfWeek'] = df['Timestamp'].dt.dayofweek.astype('category')
        df['Month'] = df['Timestamp'].dt.month.astype('category')
        
        # FIX: The variable 'x' is already the integer hour, so no .hour attribute is needed.
        df['IsBusinessHours'] = df['Hour'].apply(lambda x: 1 if 8 <= x < 18 else 0).astype('category')
        
        df.drop(columns=['Timestamp'], inplace=True)
        return df

    df_train = initial_clean(df_train)
    df_val = initial_clean(df_val)
    
    print(f"Initial clean and temporal features complete. Train size: {len(df_train)}, Val size: {len(df_val)}")

    # --- 2. Frequency Encoding Calculation (ONLY on df_train) ---

    # Identify columns remaining for frequency encoding
    freq_encode_cols = [
        col for col in df_train.columns 
        if col not in OHE_COLS and col != 'IncidentGrade' and df_train[col].dtype in ['object', 'category', 'int64', 'int32']
    ]

    for col in freq_encode_cols:
        # Calculate the map on the TRAINING data
        counts = df_train[col].value_counts().to_dict()
        freq_maps[col] = counts 

    # --- 3. Applying Frequency Encoding (To BOTH DataFrames) ---

    def apply_freq_encoding(df):
        for col, counts_map in freq_maps.items():
            # Apply map and fill unseen values (NaNs) with 0.0 (zero frequency)
            df[f'{col}_Freq'] = df[col].map(counts_map).fillna(0.0).astype('float32')
            df.drop(columns=[col], inplace=True, errors='ignore')
        
        # Ensure remaining categorical types (from initial cleaning) are converted to float32
        for col in df.columns:
            if df[col].dtype in ['bool', 'int64', 'int32']:
                df[col] = df[col].astype('float32')
        return df

    df_train = apply_freq_encoding(df_train)
    df_val = apply_freq_encoding(df_val)
    print("Frequency Encoding applied to both datasets using training maps.")


    # --- 4. One-Hot Encoding and Target Encoding ---

    # Concatenate to ensure OHE creates the exact same columns for both sets
    df_combined = pd.concat([df_train.drop(columns='IncidentGrade'), 
                             df_val.drop(columns='IncidentGrade')], 
                            keys=['train', 'val'])
    
    df_combined = pd.get_dummies(df_combined, columns=OHE_COLS, drop_first=False)

    # Separate targets
    y_train_raw = df_train['IncidentGrade']
    y_val_raw = df_val['IncidentGrade']
    # Clean up memory
    del df_train
    del df_val
    gc.collect()
    # Separate back into train/val sets
    df_train_encoded = df_combined.loc['train'].copy()
    df_val_encoded = df_combined.loc['val'].copy()

    # --- 5. Final Alignment and Target Encoding ---
    le = LabelEncoder()
    grade_order = ['FalsePositive', 'BenignPositive', 'TruePositive'] 
    le.fit(grade_order)

    # Final cleanup and target creation
    df_train_encoded['IncidentGrade_Encoded'] = le.transform(y_train_raw)
    df_val_encoded['IncidentGrade_Encoded'] = le.transform(y_val_raw)
    
    # Ensure all feature columns are explicitly float32 for XGBoost consistency
    train_cols_to_convert = [col for col in df_train_encoded.columns if col != 'IncidentGrade_Encoded']
    df_train_encoded[train_cols_to_convert] = df_train_encoded[train_cols_to_convert].astype('float32')
    df_val_encoded[train_cols_to_convert] = df_val_encoded[train_cols_to_convert].astype('float32')
    
    # Save artifacts for later use
    joblib.dump(freq_maps, 'joblibs/frequency_maps.joblib')
    joblib.dump(le, 'joblibs/target_label_encoder.joblib')
    joblib.dump(df_train_encoded.drop(columns=['IncidentGrade_Encoded']).columns.tolist(), 'joblibs/training_columns.joblib')

    return df_train_encoded, df_val_encoded


# --- 2. Load Data and Execute the Pipeline ---

df_train = pd.read_csv("./data/GUIDE_Train.csv", low_memory=False)
df_val = pd.read_csv("./data/GUIDE_Test.csv", low_memory=False)

df_train_encoded, df_val_encoded = transform_data(df_train, df_val)



print("\n--- Feature Engineering Pipeline Concluded ---")
print(f"Final Train Encoded Shape: {df_train_encoded.shape}")
print(f"Final Val Encoded Shape: {df_val_encoded.shape}")
print("Artifacts (maps and encoders) saved successfully. Ready for modeling.")

Initial clean and temporal features complete. Train size: 9465497, Val size: 4147992
Frequency Encoding applied to both datasets using training maps.

--- Feature Engineering Pipeline Concluded ---
Final Train Encoded Shape: (9465497, 130)
Final Val Encoded Shape: (4147992, 130)
Artifacts (maps and encoders) saved successfully. Ready for modeling.


Trainig

In [ ]:
X_train = df_train_encoded.drop(columns=['IncidentGrade_Encoded'])
y_train = df_train_encoded['IncidentGrade_Encoded']

X_val = df_val_encoded.drop(columns=['IncidentGrade_Encoded'])
y_val = df_val_encoded['IncidentGrade_Encoded']

del df_train_encoded
del df_val_encoded
gc.collect()

print("✅ Data separated into X/y train and validation sets.")
print(f"X_train shape: {X_train.shape}")
print(f"X_val shape: {X_val.shape}")

✅ Data separated into X/y train and validation sets.
X_train shape: (9465497, 129)
X_val shape: (4147992, 129)


In [ ]:
import xgboost as xgb
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import accuracy_score, confusion_matrix

model_final = xgb.XGBClassifier(
    objective='multi:softprob',
    num_class=3,
    n_estimators=500,                  
    device='cuda',                      
    random_state=42,
    eval_metric='mlogloss',
    n_jobs=-1,
    early_stopping_rounds=50            
)

print("\nStarting FINAL model training on clean data...")
model_final.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)], 
    verbose=False
)
print("Training complete.")


Starting FINAL model training on clean data...
Training complete.


In [ ]:
best_n_estimators = model_final.best_iteration

y_val_pred = model_final.predict(X_val)

print("\n--- FINAL MODEL PERFORMANCE ON UNSEEN VALIDATION DATA ---")
print(f"Optimal number of trees found: {best_n_estimators}")
print(f"Validation Accuracy: {accuracy_score(y_val, y_val_pred):.4f}")
print(f"Validation Cohen's Kappa (QWK): {cohen_kappa_score(y_val, y_val_pred, weights='quadratic'):.4f}")
print("Confusion Matrix (Rows=Actual, Columns=Predicted):\n", confusion_matrix(y_val, y_val_pred))

/home/popoy/miniconda3/envs/proj_1/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [00:57:53] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)



--- FINAL MODEL PERFORMANCE ON UNSEEN VALIDATION DATA ---
Optimal number of trees found: 334
Validation Accuracy: 0.8717
Validation Cohen's Kappa (QWK): 0.8412
Confusion Matrix (Rows=Actual, Columns=Predicted):
 [[1665510   46899   40531]
 [ 146358  622431  133909]
 [ 133533   30874 1327947]]


In [ ]:

joblib.dump(model_final, 'joblibs/xgboost_incident_grade_model.joblib')

print("✅ FINAL Optimized Model saved successfully to 'xgboost_incident_grade_model.joblib'.")

✅ FINAL Optimized Model saved successfully to 'xgboost_incident_grade_model.joblib'.
